In [1]:
using Revise

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/Revise.ji for module Revise.


In [29]:
using Plots; gr()
using JLD2
using FileIO
using DataFrames
using CSV

INFO: Precompiling module CSV.


In [6]:
import LCPSim
import LearningMPC
import Hoppers
import Nets

In [7]:
data = load("hopper-0.5.jld2")

Dict{String,Any} with 2 entries:
  "snapshots" => LearningMPC.Snapshot{Float64}[LearningMPC.Snapshot{Float64}([1…
  "dataset"   => LearningMPC.Dataset{Float64}(LearningMPC.LQRSolution, Learning…

In [23]:
sample = data["dataset"].training_data[1]

LearningMPC.Sample{Float64}([0.25, 0.25, -0.32893, -0.32893], [-0.0 -5.54794e-14 … 2.91496e-15 2.99889e-29; 40.0 3.80482e-14 … 9.29637e-15 5.0863e-22], [925.549, 386.659], LearningMPC.MIPResults
  solvetime_s: Float64 0.03131890296936035
  objective_value: Float64 159.11890792096466
  objective_bound: Float64 143.46709466893594
)

INFO: Precompiling module DataFrames.


In [24]:
function Base.convert(DataFrame, samples::AbstractArray{<:LearningMPC.Sample})
    df = DataFrame()
    s = first(samples)
    for i in 1:length(s.state)
        df[Symbol("x$i")] = [s.state[i] for s in samples]
    end
    for i in 1:size(s.uJ, 1)
        df[Symbol("u$i")] = [s.uJ[i, 1] for s in samples]
    end
    for i in 1:size(s.uJ, 1)
        for j in 2:size(s.uJ, 2)
            df[Symbol("J_{$i, $(j-1)}")] = [s.uJ[i, j] for s in samples]
        end
    end
    df
end

In [30]:
train_df = convert(DataFrame, data["dataset"].training_data)

,x1,x2,x3,x4,u1,u2,"J_{1, 1}","J_{1, 2}","J_{1, 3}","J_{1, 4}","J_{2, 1}","J_{2, 2}","J_{2, 3}","J_{2, 4}"
1,0.25,0.25,-0.32893,-0.32893,-0.0,40.0,-5.54794e-14,-1.20671e-13,2.91496e-15,2.99889e-29,3.80482e-14,9.04612e-14,9.29637e-15,5.0863e-22
2,0.309028,0.309028,1.18057,1.18057,-0.0,40.0,-6.65185e-14,-4.48046e-14,2.57747e-15,-3.71337e-31,-2.27359e-14,1.66328e-13,-4.82253e-15,-1.29121e-21
3,0.443532,0.443532,2.69007,2.69007,-0.0,33.4645,1.61717e-15,8.65675e-17,7.02041e-16,8.33646e-22,-51.6519,7.67282,-19.8464,-1.31783e-7
4,0.81305,0.81305,3.51757,3.51756,-0.0,14.2384,1.90738e-14,3.07615e-20,-1.23023e-15,-1.46652e-21,-399.965,-0.00313,-20.0,-1.23018e-11
5,1.36271,1.0,1.92758,0.0,-0.0,1.02319,1.02621e-13,2.26742e-14,7.13539e-14,5.59762e-15,1.76802e-6,-32.5675,8.2928e-7,-1.62838
6,1.43457,1.0,1.43708,0.0,-0.0,1.02319,-1.05992e-11,5.23813e-13,5.65354e-13,-2.2078e-14,6.31621e-5,-32.5675,2.53079e-5,-1.62838
7,1.50298,1.0,-0.0344179,0.0,-0.0,-0.381561,1.02143e-12,-9.61392e-15,-1.8539e-14,-8.65223e-14,2.21398e-5,-36.3636,5.56771e-6,-1.81818
8,1.42596,1.0,-0.99634,0.209859,-0.0,-0.742314,-1.04946e-13,-1.3508e-14,1.13039e-14,1.80624e-15,1.10237e-6,-36.3636,1.78446e-7,-1.81818
9,1.1265,1.0,-2.48692,0.0,-0.0,-3.30637,-3.77836e-14,2.68569e-14,3.54339e-15,2.75584e-16,13.3447,-18.0138,0.624731,-1.87176
10,0.969362,0.909075,-3.14274,-1.8185,-0.0,14.7357,1.68547e-16,-2.13048e-20,5.69196e-16,9.4935e-22,-79.9999,3.90602e-5,-20.0,-7.70202e-13


In [32]:
CSV.write("training.csv", train_df)
CSV.write("validation.csv", convert(DataFrame, data["dataset"].validation_data))
CSV.write("testing.csv", convert(DataFrame, data["dataset"].testing_data))


CSV.Sink{DateFormat{Symbol("yyyy-mm-dd"),Tuple{Base.Dates.DatePart{'y'},Base.Dates.Delim{Char,1},Base.Dates.DatePart{'m'},Base.Dates.Delim{Char,1},Base.Dates.DatePart{'d'}}},DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd"
        decimal: '.'
        truestring: 'true'
        falsestring: 'false', IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "testing.csv", 90, true, String["x1", "x2", "x3", "x4", "u1", "u2", "J_{1, 1}", "J_{1, 2}", "J_{1, 3}", "J_{1, 4}", "J_{2, 1}", "J_{2, 2}", "J_{2, 3}", "J_{2, 4}"], 14, false, Val{false})